In [1]:
import os
import ast
import openai
import time
import json
from tqdm import tqdm
from json import JSONDecodeError
import re

In [1]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = ""
# openai.Model.list()

# # engines = openai.Engine.list()
# # print(engines)

In [3]:
with open('human_summarization_by_gpt35.json', 'r') as f:
    data = json.load(f)
# with open('human2_summary_by_gpt35.json', 'r') as f:
#     data2 = json.load(f)

# data = data1 + data2
print(len(data))

1200


In [4]:
with open('evidence_corpus.json', 'r', encoding='utf-8') as f:
    evidence_data = json.load(f)
print(len(evidence_data))

8096


In [5]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
#                                  model= "gpt-3.5-turbo-16k",
                                 model= "gpt-3.5-turbo-0125",
                                 temperature=0, 
                                 max_tokens=256):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        format={"type": "json_object"}
    )
    return response.choices[0].message["content"]

In [6]:
# delimiter_1 = "#"
# delimiter_2 = "##"

# system_message = f"""
# You are a Fact Checker.\
# Given a news paragraph and evidence, assess the factual accuracy of the news story.\
# Provide a comprehensive explanation step by step and include it in the reason:\

# 1. Initial Analysis: Summarize the news paragraph's central claim, identifying key concepts.\
# 2. Alignment Check: Examine evidence for alignment with the news paragraph. Flag any discrepancies.\
# 3. Causation Assessment: Evaluate if the news paragraph confuses correlation with causation. Identify potential logical fallacies.\
# 4. Accuracy Verification:Verify quantitative and qualitative accuracy in the news paragraph compared to evidence.\
# 5. Precision Check: Assess if the news paragraph overgeneralizes or oversimplifies findings from evidence.\
# 6. Contextual Insight: Consider the broader context surrounding the news and evidence.\
# 7. Phrase Selection: Select precise phrases from both the news story and evidence to support each analysis step.\

# The user will input the news paragraph behind {delimiter_1} and the relevant evidence behind {delimiter_2}.
# """

In [7]:
delimiter_1 = "#"
delimiter_2 = "##"

system_message = f"""
You are a Fact Checker.
Given a news paragraph and evidence, assess the factual accuracy of the news story.
Make a prediction and provide an explanation for your prediction based on the defined axes of scientific validity. 

Axes of scientific validity:
1. Alignment: Alignment is defined as news and evidence representing the same meaning about one scientific content.
2. Causation Confusion: The news article may confuse correlations presented in the scientific literature as causation. This could be one dimension in which the scientific validity is compromised.
3. Accuracy: This refers to how accurately the news item describes the scientific findings quantitatively and qualitatively.
4. Generalization: This refers to overgeneralization or oversimplification of the findings reported in the scientific literature.
5. Contextual Fidelity: Does the news article retain the broader context of the scientific finding?

The user will input the news paragraph behind {delimiter_1} and the relevant evidence behind {delimiter_2}.
"""

In [8]:
i = 0
generated_result = []
all_resp = []

In [10]:
for item in tqdm(data):
#     print(f"Predicting {i}")
    
    news = item['abstractive']
    #print(news)
    
    evidence_list = []
    if type(item['evidence'])==str:
        evidence_list.append(item['evidence'])
    else:
        evidence_list.extend(item['evidence'])
#     print(f"evidence list: {evidence_list}")
    
    evidence = [it['abstract'] for it in evidence_data if it['cord_uid'] in evidence_list]
#     if evidence:
#         evidence = evidence[0]
#     else:
#         evidence = ""
    evidence = ''.join(evidence)
    
    user_message = f"""
     Here is one news paragraph {delimiter_1}{news} and relevant evidence {delimiter_2}{evidence}. 
     Determine if the evidence supports or refutes the news point of view. 
     Make a final prediction of 'True' if the news paragraph is factually true or 'False' if it is false based on the evidence under the keyword: 'prediction'. 
     Provide a comprehensive explanation for your prediction based on the defined axes of scientific validity under the keyword: 'reason'. 
     Then, score the news article based on each axis of scientific validity between [-1, 1] under the keyword: 'scores'. 
     For scoring, assign a float value in the range -1 and 1 to each axis, where -1 indicates strong disagreement, 0 indicates neutrality, and 1 indicates strong agreement.
     Provide the final output in JSON format containing the following three keys: prediction, reason, and scores.
    """
    
    assistant_message = """
    As a Fact Checker, carefully analyze the evidence abstracts and provide the result.
    Reference specific phrases or sentences from both the news story and the evidence to support your prediction.
    """
        
    messages =  [ {'role':'system', 'content': system_message},    
                  {'role':'user', 'content': user_message},  
                  {'role': 'assistant', 'content': assistant_message}]    
    
    response = get_completion_from_messages(messages)
#     print(response)
    
    result = dict()
    result['news'] = news
    result['evidence list'] = evidence_list
    result['ground truth'] = item['label']
    
    try:
        result.update(json.loads(response))
        
    except JSONDecodeError as e:
        pattern1 = r'"prediction": "(.*?)",\s*"reason": "(.*?)",\s*"scores": "(.*?)"'
        pattern2 = r"Prediction:\s*(True|False)\nReason:\s*(.*)\nReason:\s*(.*)"
        match1 = re.search(pattern1, response)
        match2 = re.search(pattern2, response)

        if match1:
            result['prediction'] = match1.group(1)
            result['reason'] = match1.group(2)
            result['scores'] = match1.group(3)
        
        elif match2:
            result['prediction'] = match2.group(1)
            result['reason'] = match2.group(2)
            result['scores'] = match1.group(3)
        
        else:
            pattern_prediction1 = r'##prediction\n(.*?)\n\n##reason'
            pattern_reason1 = r'##reason\n(.*?)\n\n##scores'
            pattern_scores1 = r'##scores\n(.*?)$'
            
            prediction_match = re.search(pattern_prediction1, response, re.DOTALL)
            reason_match = re.search(pattern_reason1, response, re.DOTALL)
            scores_match = re.search(pattern_scores1, response, re.DOTALL)

            # Extracting information
            if prediction_match and reason_match and scores_match:
                result['prediction'] = re.search(pattern_prediction1, response, re.DOTALL).group(1).strip()
                result['reason'] = re.search(pattern_reason1, response, re.DOTALL).group(1).strip()
                result['scores'] = re.search(pattern_scores1, response, re.DOTALL).group(1).strip()

            else:
                prediction_pattern = r'##prediction##\n(.*?)\n\n'
                reason_pattern = r'##reason##\n(.*?)\n\n'
                scores_pattern = r'##scores##\n(.*?)$'

                prediction_match = re.search(prediction_pattern, response, re.DOTALL)
                reason_match = re.search(reason_pattern, response, re.DOTALL)
                scores_match = re.search(scores_pattern, response, re.DOTALL)

                if prediction_match and reason_match and scores_match:
                    result['prediction'] = prediction_match.group(1)
                    result['reason'] = reason_match.group(1)
                    result['scores'] = scores_match.group(1)
                    
                    
    generated_result.append(result)
    all_resp.append(response)
    
    time.sleep(2)
    i += 1
    
#     break

  1%|          | 7/1200 [01:10<3:21:32, 10.14s/it]


APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)

In [19]:
i

5

In [20]:
len(generated_result)

5

In [21]:
result

{'news': 'The U.S. Food and Drug Administration (FDA) has changed its guidelines for COVID-19 vaccination, now only requiring one mRNA injection and no additional boosters. This change comes after over 230 million Americans have already received at least two doses of the vaccine. The FDA claims that the new "bivalent" injections are sufficient and work better against the Omicron variant, despite a lack of evidence. Individuals who have already received more than one dose may not be eligible for further booster shots. Government official Tony Fauci has also advised Americans to skip the second COVID-19 jab dose. These changes come as demand for mRNA injections has decreased and more people are becoming aware of the potential risks.',
 'evidence list': ['yhun6hid', '3rxtk8as', '4exns0xv'],
 'ground truth': 0}

In [14]:
result['prediction'] = 'False'
result['reason'] = "The news paragraph is factually false based on the evidence provided. Here is a step-by-step analysis:\n\n1. Initial Analysis: The central claim of the news paragraph is that singer Celine Dion developed Stiff Person Syndrome (SPS) as a rare adverse effect of COVID-19 vaccination.\n\n2. Alignment Check: The evidence provided does not mention anything about Celine Dion or Stiff Person Syndrome. There is no alignment between the news paragraph and the evidence.\n\n3. Causation Assessment: The news paragraph suggests a causal link between the Pfizer mRNA vaccine and SPS, but there is no evidence provided to support this claim. It is important to note that correlation does not imply causation.\n\n4. Accuracy Verification: The news paragraph claims that Pfizer was aware of the link between their vaccine and SPS in early 2021, but there is no evidence provided to support this claim. The evidence only discusses adverse events following COVID-19 immunization without mentioning SPS.\n\n5. Precision Check: The news paragraph overgeneralizes by stating that SPS is a side effect of Pfizer's mRNA vaccine without providing specific evidence to support this claim.\n\n6"

In [32]:
prediction_start = response.find('"prediction": "') + len('"prediction": "')
prediction_end = response.find('"', prediction_start)
reason_start = response.find('"reason": "') + len('"reason": "')
reason_end = response.find('"', reason_start)

# Extract the prediction and reason values
prediction = response[prediction_start:prediction_end]
reason = response[reason_start:reason_end]

In [34]:
prediction

'False'

In [39]:
generated_result[-1]

{'news': "Singer Celine Dion has reportedly developed Stiff Person Syndrome (SPS), a rare adverse effect of COVID-19 vaccination. SPS, characterized by rigidity and muscle spasms, is listed as a side effect of Pfizer's mRNA vaccine. Pfizer was aware of the link between their vaccine and SPS in early 2021, but the FDA did not disclose this information to the public. Data from the Vaccine Adverse Event Reporting System (VAERS) suggests a 99.6% chance that SPS is a direct result of COVID-19 vaccination. Dion's existing health issues may have made her more susceptible to developing SPS. The timing of her symptoms could indicate a direct adverse effect of the vaccine. These findings raise concerns about the safety of COVID-19 vaccines.",
 'evidence list': ['znt1jdgx', 'gljvjske', 'gaesasxq'],
 'ground truth': 0,
 'prediction': 'False',
 'reason': "The news paragraph is factually false based on the evidence provided. Here is a step-by-step analysis:\\n\\n1. Initial Analysis: The central clai

In [13]:
# generated_result
# for item in data:
# #     print(item['abstractive'])
#     print(item)
#     break

In [14]:
# data[537]['prediction'] = "refute"
# data[537]['reason'] = "The relevant sentences refute the point of view presented in the news paragraph. The news paragraph claims that there is evidence that cancers are occurring in ways never before seen after people receive COVID-19 vaccinations, and that the Covid-19 vaccine has compromised immune systems by disabling the body's ability to fight non-normal cancerous cells. However, none of the relevant sentences mention anything about cancers occurring in new ways or the Covid-19 vaccine compromising immune systems. Instead, the relevant sentences discuss formalisation in Isabelle/HOL, wider array of vascular phenomena related to Covid vaccinations, and the belief that a healthy immune system can fight and disable cancerous cells. These sentences do not provide any supporting evidence for the claims made in the news paragraph, and therefore refute its point of view."

In [15]:
with open('explanation_cot_prediction_GPT35_human.json', 'w') as file:
    json.dump(generated_result, file)

In [26]:
# data[515]

In [76]:
j = 0
for i in generated_result:
    if 'prediction' not in i.keys():
        print(j)
        print(i,end='\n\n\n')
    j += 1
#     break

In [75]:
generated_result[121].keys()

dict_keys(['news', 'evidence list', 'ground truth', 'prediction', 'reason'])

In [23]:
generated_result[134]

{'news': "Singer Celine Dion has reportedly developed Stiff Person Syndrome (SPS), a rare adverse effect of COVID-19 vaccination. SPS, characterized by rigidity and muscle spasms, is listed as a side effect of Pfizer's mRNA vaccine. Pfizer was aware of the link between their vaccine and SPS in early 2021, but the FDA did not disclose this information to the public. Data from the Vaccine Adverse Event Reporting System (VAERS) suggests a 99.6% chance that SPS is a direct result of COVID-19 vaccination. Dion's existing health issues may have made her more susceptible to developing SPS. The timing of her symptoms could indicate a direct adverse effect of the vaccine. These findings raise concerns about the safety of COVID-19 vaccines.",
 'evidence list': ['znt1jdgx', 'gljvjske', 'gaesasxq'],
 'ground truth': 0}

In [ ]:
Predicting 121
evidence list: ['jyuby0bt', 'bewzupb6', 'ph3bv7vc']
{
  "prediction": "False",
  "reason": "

In [73]:
generated_result[121]['prediction'] = 'False'
generated_result[121]['reason'] = "The news paragraph contains several inaccuracies and misinterpretations of the evidence. Firstly, the news paragraph claims that lipid nanoparticles (LNPs) in COVID-19 vaccines are the true pathogen, which is not supported by the evidence. The evidence discusses the use of LNPs as a delivery vehicle for mRNA vaccines, highlighting their role in preventing enzymatic degradation and enhancing the potency of the vaccines. There is no mention of LNPs being the true pathogen or causing disseminated intravascular coagulation (DIC) or immune suppression. Secondly, the news paragraph suggests that the Novavax COVID-19 shot should be avoided due to potential LNP content, but the evidence does not mention Novavax or any specific COVID-19 vaccine. Thirdly, the news paragraph claims that two ingredients in mRNA vaccines, DSPC and DSPE, have been patented to bypass the blood-brain barrier, but the evidence does not support this claim. The evidence discusses the surface modification of LNPs with mannans to enhance antibody response, but there is no mention of bypassing the blood-brain barrier. Fourthly, the news paragraph disputes the safety of COVID-19 shots for pregnant women based on the presence of mRNA and"

In [54]:
for item in data[916:]:
    print(item['abstractive'])
#     print(item['evidence'])
#     print(item['value'])
    break

The CDC has confirmed that by October 2022, 58,000 children in the USA were injured, 15,000 were hospitalized, 1,201 were left disabled, and 163 tragically died due to COVID-19 vaccination. The CDC's Advisory Committee voted in favor of adding the COVID-19 vaccine to the recommended immunization schedule for children aged 6 months and over. However, it is unclear if the committee was aware of these figures before making their decision. The CDC's Vaccine Adverse Event Reporting System (VAERS) reveals that the Pfizer, Moderna, and Janssen vaccines have caused injuries and hospitalizations among children. The CDC estimates that only 1 to 10% of adverse events are reported to VAERS, so the true consequences of COVID-19 vaccination among children may be higher. The article also appeals for support to continue bringing uncensored news and investigative journalism.
